In [1]:
from simpleai.search import CspProblem, backtrack, hill_climbing, simulated_annealing, genetic
from simpleai.search.csp import min_conflicts
import random

# Định nghĩa các heuristic cho Backtracking
# Heuristic biến: chọn biến có ít giá trị nhất (most constrained variable)
def most_constrained_variable(variables, domains, constraints, assigned):
    # Lọc ra các biến chưa được gán
    unassigned = [v for v in variables if v not in assigned]
    # Trả về biến có miền giá trị (domain) nhỏ nhất
    return min(unassigned, key=lambda var: len(domains[var]))

# Heuristic giá trị: chọn giá trị ít gây ràng buộc nhất (least constraining value)
def least_constraining_value(variable, domains, constraints, assigned):
    # Hàm con để đếm số ràng buộc bị ảnh hưởng nếu chọn một giá trị
    def count_conflicts(value):
        count = 0
        # Duyệt qua tất cả các biến chưa gán
        for var in domains:
            if var != variable and var not in assigned:
                # Duyệt qua tất cả các giá trị của biến đó
                for val in domains[var]:
                    # Kiểm tra xem ràng buộc có bị vi phạm không
                    if not constraints.get((min(variable, var), max(variable, var)), lambda v1, v2, v3, v4: True)(
                        (variable, var), (value, val)
                    ):
                        count += 1
        return count
    values = domains[variable]
    # Sắp xếp các giá trị theo thứ tự tăng dần của số xung đột
    return sorted(values, key=count_conflicts)

# Định nghĩa bài toán 5 hậu dưới dạng CSP (dùng cho Backtracking)
def create_5_queens_csp():
    variables = [f"Q{i}" for i in range(5)]
    domains = {v: list(range(5)) for v in variables}
    constraints = []
    
    for i in range(5):
        for j in range(i + 1, 5):
            v1, v2 = f"Q{i}", f"Q{j}"
            
            # Hàm tạo ra hàm ràng buộc, đóng vai trò "closure" để lưu lại i và j
            def make_constraint(i=i, j=j):
                # Hàm ràng buộc thực tế, nhận tuple (biến, giá trị) từ simpleai
                def constraint(variables_tuple, values_tuple):
                    # Giải nén tuple giá trị
                    val_i, val_j = values_tuple
                    # Kiểm tra xung đột trên hàng
                    if val_i == val_j:
                        return False
                    # Kiểm tra xung đột trên đường chéo
                    return abs(i - j) != abs(val_i - val_j)
                return constraint
            
            constraints.append(((v1, v2), make_constraint()))
    
    return CspProblem(variables, domains, constraints)

# Định nghĩa bài toán 5 hậu cho Local Search bằng cách kế thừa CspProblem
class CspLocalSearchProblem(CspProblem):
    def __init__(self, variables, domains, constraints):
        super().__init__(variables, domains, constraints)
        # Khởi tạo trạng thái ngẫu nhiên ban đầu cho local search
        self.initial_state = {var: random.choice(self.domains[var]) for var in self.variables}
    
    # Phương thức `value` là hàm heuristic mà local search cố gắng tối đa hóa
    def value(self, state):
        conflicts = 0
        # Duyệt qua tất cả các cặp hậu để đếm xung đột
        for i in range(len(self.variables)):
            for j in range(i + 1, len(self.variables)):
                v1, v2 = self.variables[i], self.variables[j]
                val_i, val_j = state[v1], state[v2]
                
                # Kiểm tra xung đột trên hàng và đường chéo
                if val_i == val_j or abs(i - j) == abs(val_i - val_j):
                    conflicts += 1
        # Trả về giá trị âm của xung đột, vì simpleai tìm giá trị tối đa
        return -conflicts

    # Phương thức `actions` trả về tất cả các hành động có thể từ một trạng thái
    def actions(self, state):
        actions_list = []
        # Hành động là thay đổi vị trí của một hậu
        for var in self.variables:
            for val in self.domains[var]:
                if state[var] != val:
                    actions_list.append((var, val))
        return actions_list

    # Phương thức `result` trả về trạng thái mới sau khi thực hiện một hành động
    def result(self, state, action):
        new_state = state.copy()
        var, val = action
        new_state[var] = val
        return new_state

    # Phương thức `cost` trả về chi phí của một hành động
    def cost(self, state1, action, state2):
        # Chi phí của mỗi hành động (di chuyển 1 hậu) là 1
        return 1

    # Phương thức `generate_random_state` để tạo ra một trạng thái ngẫu nhiên hoàn chỉnh
    # Cần thiết cho các thuật toán khởi tạo với nhiều trạng thái ngẫu nhiên (như Genetic Algorithm)
    def generate_random_state(self):
        return {var: random.choice(self.domains[var]) for var in self.variables}
    
    # Phương thức `crossover` để lai ghép hai trạng thái cha mẹ
    def crossover(self, parent1_state, parent2_state):
        # Chọn ngẫu nhiên một điểm lai ghép
        crossover_point = random.randint(1, len(self.variables) - 1)
        child_state = {}
        
        # Lấy một phần của trạng thái từ cha mẹ 1
        for i in range(crossover_point):
            var = self.variables[i]
            child_state[var] = parent1_state[var]
            
        # Lấy phần còn lại của trạng thái từ cha mẹ 2
        for i in range(crossover_point, len(self.variables)):
            var = self.variables[i]
            child_state[var] = parent2_state[var]
            
        return child_state
        
    # Phương thức `mutate` để đột biến một trạng thái
    def mutate(self, state):
        mutated_state = state.copy()
        # Chọn ngẫu nhiên một biến (hậu) để đột biến
        var = random.choice(self.variables)
        # Chọn ngẫu nhiên một giá trị (hàng) mới cho biến đó
        new_val = random.choice(self.domains[var])
        mutated_state[var] = new_val
        return mutated_state

# Hàm chạy backtracking với các tùy chọn heuristic và AC-3
def run_backtracking(ac3=False, var_heuristic=None, val_heuristic=None):
    problem = create_5_queens_csp()
    # inference=True sẽ kích hoạt AC-3
    result = backtrack(problem,
                       variable_heuristic=var_heuristic,
                       value_heuristic=val_heuristic,
                       inference=ac3)
    return result

# Hàm chạy local search (GA, SA, Hill Climbing)
def run_local_search(method='hill_climbing'):
    # Sử dụng lớp tùy chỉnh cho local search
    problem = CspLocalSearchProblem(
        variables=[f"Q{i}" for i in range(5)],
        domains={v: list(range(5)) for v in [f"Q{i}" for i in range(5)]},
        constraints=[] # Các ràng buộc không cần thiết cho local search
    )
    
    if method == 'hill_climbing':
        result = hill_climbing(problem)
    elif method == 'simulated_annealing':
        result = simulated_annealing(problem)
    elif method == 'genetic':
        # Sửa generations_limit thành iterations_limit để phù hợp với thư viện
        result = genetic(problem, population_size=50, iterations_limit=100)
    else:
        raise ValueError("Phương pháp không hợp lệ")
    
    return result.state

# Chạy thử nghiệm và in kết quả
if __name__ == '__main__':
    print("Backtracking không AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value")
    sol1 = run_backtracking(ac3=False,
                           var_heuristic=most_constrained_variable,
                           val_heuristic=least_constraining_value)
    print(sol1)

    print("\nBacktracking có AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value")
    sol2 = run_backtracking(ac3=True,
                           var_heuristic=most_constrained_variable,
                           val_heuristic=least_constraining_value)
    print(sol2)

    print("\nLocal search - Hill Climbing")
    sol3 = run_local_search('hill_climbing')
    print(sol3)

    print("\nLocal search - Simulated Annealing")
    sol4 = run_local_search('simulated_annealing')
    print(sol4)

    print("\nLocal search - Genetic Algorithm")
    sol5 = run_local_search('genetic')
    print(sol5)

Backtracking không AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value
{'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}

Backtracking có AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value
{'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}

Local search - Hill Climbing
{'Q0': 3, 'Q1': 0, 'Q2': 2, 'Q3': 4, 'Q4': 1}

Local search - Simulated Annealing
{'Q0': 4, 'Q1': 4, 'Q2': 4, 'Q3': 2, 'Q4': 2}

Local search - Genetic Algorithm
{'Q0': 3, 'Q1': 1, 'Q2': 2, 'Q3': 2, 'Q4': 2}
